# ಮಾದರಿ 06: ಬುದ್ಧಿವಂತ ಮಾದರಿ ರೌಟರ್ - ಸಾಧನಗಳಾಗಿ ಮಾದರಿಗಳು

ಈ ನೋಟ್ಬುಕ್ ವಿವಿಧ ರೀತಿಯ ಕಾರ್ಯಗಳಿಗೆ ಅತ್ಯಂತ ಸೂಕ್ತವಾದ AI ಮಾದರಿಯನ್ನು ಸ್ವಯಂಚಾಲಿತವಾಗಿ ಆಯ್ಕೆಮಾಡುವ ಬುದ್ಧಿವಂತ ರೌಟಿಂಗ್ ವ್ಯವಸ್ಥೆಯನ್ನು ಪ್ರದರ್ಶಿಸುತ್ತದೆ, Microsoft Foundry Local ನೊಂದಿಗೆ "ಮಾದರಿಗಳನ್ನು ಸಾಧನಗಳಾಗಿ" ಪರಿಕಲ್ಪನೆಯನ್ನು ತೋರಿಸುತ್ತದೆ.

## ಅವಲೋಕನ

ಮಾದರಿ ರೌಟರ್ ಬುದ್ಧಿವಂತ ಕಾರ್ಯ ವರ್ಗೀಕರಣ ಮತ್ತು ಮಾದರಿ ಆಯ್ಕೆಗಳನ್ನು ಅನುಷ್ಠಾನಗೊಳಿಸುತ್ತದೆ:

- 🎯 **ಕಾರ್ಯ ವರ್ಗೀಕರಣ**: ಬಳಕೆದಾರ ಪ್ರಶ್ನೆಗಳನ್ನು ಸ್ವಯಂಚಾಲಿತವಾಗಿ ವರ್ಗೀಕರಿಸುತ್ತದೆ
- 🔧 **ಮಾದರಿ ಆಯ್ಕೆ**: ಕಾರ್ಯಗಳನ್ನು ವಿಶೇಷ ಮಾದರಿಗಳೊಂದಿಗೆ ಹೊಂದಿಸುತ್ತದೆ
- ⚙️ **ಡೈನಾಮಿಕ್ ಸಂರಚನೆ**: ಪರಿಸರ ಆಧಾರಿತ ಸಾಧನ ರಿಜಿಸ್ಟ್ರಿ
- 📊 **ಆರೋಗ್ಯ ಮೇಲ್ವಿಚಾರಣೆ**: ಸೇವಾ ಸ್ಥಿತಿ ಮತ್ತು ಮಾದರಿ ಲಭ್ಯತೆ
- 🚀 **ಉತ್ಪಾದನಾ ಸಿದ್ಧತೆ**: ಉದ್ಯಮ-ಮಟ್ಟದ ರೌಟಿಂಗ್ ಮತ್ತು ದೋಷ ನಿರ್ವಹಣೆ


## ವಾಸ್ತುಶಿಲ್ಪ ಅವಲೋಕನ

```
User Query → Task Classifier → Model Selector → Specialized Model → Response
     ↓              ↓               ↓              ↓
"Fix this code" → Code Task → qwen2.5-7b → Code Model → Fixed Code
"Why is...?"   → Reasoning  → deepseek-r1 → Reasoning → Analysis
"Write story"  → Creative   → phi-4-mini  → Creative → Story
```


## ಪೂರ್ವಾಪೇಕ್ಷೆಗಳು ಮತ್ತು ಸೆಟಪ್

ಅತ್ಯುತ್ತಮ ಮಾರ್ಗದರ್ಶನಕ್ಕಾಗಿ, ನೀವು ಹಲವಾರು ಮಾದರಿಗಳನ್ನು ಚಾಲನೆ ಮಾಡಿರಬೇಕು. ಪರಿಸರವನ್ನು ಸೆಟ್ ಅಪ್ ಮಾಡೋಣ:


In [ ]:
# Install required packages
!pip install openai foundry-local-sdk

## ಗ್ರಂಥಾಲಯಗಳು ಮತ್ತು ಸಂರಚನೆ ಆಮದುಮಾಡಿ


In [ ]:
import os
import sys
import re
import json
import subprocess
import time
from typing import Dict, Any, Optional, List
from openai import OpenAI

try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
    print("✅ Foundry Local SDK is available")
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False
    print("⚠️ Foundry Local SDK not available, will use manual configuration")

## ಮಾದರಿ ರೌಟರ್ ವರ್ಗ

ಬುದ್ಧಿವಂತ ಮಾದರಿ ಆಯ್ಕೆ ನಿರ್ವಹಿಸುವ ಮೂಲ ರೌಟರ್:


In [ ]:
class ModelRouter:
    """Intelligent model router that selects appropriate models for different task types."""
    
    def __init__(self):
        self.client = None
        self.base_url = None
        self.tools = self._load_tool_registry()
        self._initialize_client()
    
    def _load_tool_registry(self) -> Dict[str, Dict[str, Any]]:
        """Load tool registry from environment or use defaults."""
        default_tools = {
            "general": {
                "model": "phi-4-mini",
                "notes": "Fast general-purpose chat and Q&A",
                "temperature": 0.7
            },
            "reasoning": {
                "model": "deepseek-r1-7b",
                "notes": "Step-by-step analysis and logical reasoning",
                "temperature": 0.3
            },
            "code": {
                "model": "qwen2.5-7b",
                "notes": "Code generation, debugging, and technical tasks",
                "temperature": 0.2
            },
            "creative": {
                "model": "phi-4-mini",
                "notes": "Creative writing and storytelling",
                "temperature": 0.9
            }
        }
        
        # Override with environment variables
        if os.environ.get("GENERAL_MODEL"):
            default_tools["general"]["model"] = os.environ["GENERAL_MODEL"]
        if os.environ.get("REASONING_MODEL"):
            default_tools["reasoning"]["model"] = os.environ["REASONING_MODEL"]
        if os.environ.get("CODE_MODEL"):
            default_tools["code"]["model"] = os.environ["CODE_MODEL"]
        if os.environ.get("CREATIVE_MODEL"):
            default_tools["creative"]["model"] = os.environ["CREATIVE_MODEL"]
        
        # Check for complete JSON override
        tools_env = os.environ.get("TOOL_REGISTRY")
        if tools_env:
            try:
                custom_tools = json.loads(tools_env)
                return custom_tools
            except json.JSONDecodeError:
                print("⚠️ Invalid TOOL_REGISTRY JSON, using defaults")
        
        return default_tools
    
    def _discover_base_url(self, default: str = "http://localhost:8000") -> str:
        """Discover Foundry Local service URL."""
        env_url = os.environ.get("BASE_URL")
        if env_url:
            return env_url
        
        try:
            # Try to get URL from Foundry CLI
            result = subprocess.run(
                ["foundry", "service", "status"],
                capture_output=True, text=True, timeout=3
            )
            if result.returncode == 0:
                match = re.search(r"https?://[\w\.-]+(?::\d+)?", result.stdout)
                if match:
                    return match.group(0)
        except (subprocess.TimeoutExpired, subprocess.CalledProcessError, FileNotFoundError):
            pass
        
        return default
    
    def _initialize_client(self):
        """Initialize OpenAI client with Foundry Local or fallback configuration."""
        if FOUNDRY_SDK_AVAILABLE:
            try:
                # Try to use any available model for client initialization
                first_model = next(iter(self.tools.values()))["model"]
                print(f"🔄 Initializing Foundry Local SDK with model: {first_model}...")
                manager = FoundryLocalManager(first_model)
                
                self.client = OpenAI(
                    base_url=manager.endpoint,
                    api_key=manager.api_key
                )
                self.base_url = manager.endpoint
                print(f"✅ Foundry Local SDK initialized")
                return
            except Exception as e:
                print(f"⚠️ Could not use Foundry SDK ({e}), falling back to manual configuration")
        
        # Fallback to manual configuration
        self.base_url = self._discover_base_url()
        api_key = os.environ.get("API_KEY", "")
        
        self.client = OpenAI(
            base_url=f"{self.base_url}/v1",
            api_key=api_key
        )
        print(f"🔧 Manual configuration initialized at {self.base_url}")
    
    def check_service_health(self) -> Dict[str, Any]:
        """Check Foundry Local service health and available models."""
        try:
            # Try to list models
            models_response = self.client.models.list()
            available_models = [model.id for model in models_response.data]
            
            # Check which configured models are available
            configured_models = [tool["model"] for tool in self.tools.values()]
            available_configured = [m for m in configured_models if m in available_models]
            
            return {
                "status": "healthy",
                "base_url": self.base_url,
                "available_models": available_models,
                "configured_models": configured_models,
                "available_configured": available_configured,
                "tools_configured": list(self.tools.keys())
            }
        except Exception as e:
            return {
                "status": "error",
                "base_url": self.base_url,
                "error": str(e)
            }
    
    def select_tool(self, user_query: str) -> str:
        """Select the most appropriate tool based on the user query."""
        query_lower = user_query.lower()
        
        # Code-related keywords
        code_keywords = [
            "code", "python", "function", "class", "method", "bug", "debug", 
            "programming", "script", "algorithm", "implementation", "refactor",
            "syntax", "compile", "error", "exception", "variable", "loop"
        ]
        if any(keyword in query_lower for keyword in code_keywords):
            return "code"
        
        # Reasoning keywords
        reasoning_keywords = [
            "why", "how", "explain", "step-by-step", "reason", "analyze", 
            "think", "logic", "because", "cause", "compare", "evaluate",
            "pros and cons", "advantage", "disadvantage", "benefit", "drawback"
        ]
        if any(keyword in query_lower for keyword in reasoning_keywords):
            return "reasoning"
        
        # Creative keywords
        creative_keywords = [
            "story", "poem", "creative", "imagine", "write", "tale", 
            "narrative", "fiction", "character", "plot", "novel", "poetry",
            "song", "lyrics", "dialogue", "script"
        ]
        if any(keyword in query_lower for keyword in creative_keywords):
            return "creative"
        
        # Default to general
        return "general"
    
    def chat(self, model: str, content: str, max_tokens: int = 300, temperature: Optional[float] = None) -> str:
        """Send chat completion request to the specified model."""
        try:
            params = {
                "model": model,
                "messages": [{"role": "user", "content": content}],
                "max_tokens": max_tokens
            }
            
            if temperature is not None:
                params["temperature"] = temperature
            
            response = self.client.chat.completions.create(**params)
            return response.choices[0].message.content
        except Exception as e:
            return f"Error generating response with model {model}: {str(e)}"
    
    def route_and_run(self, prompt: str) -> Dict[str, Any]:
        """Route the prompt to the appropriate model and generate response."""
        tool_key = self.select_tool(prompt)
        tool_config = self.tools[tool_key]
        model = tool_config["model"]
        temperature = tool_config.get("temperature", 0.7)
        
        start_time = time.time()
        answer = self.chat(
            model=model, 
            content=prompt, 
            max_tokens=400, 
            temperature=temperature
        )
        end_time = time.time()
        
        return {
            "tool": tool_key,
            "model": model,
            "tool_description": tool_config["notes"],
            "temperature": temperature,
            "processing_time": end_time - start_time,
            "answer": answer,
            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
        }

# Initialize the router
print("Initializing Model Router...")
print("=" * 50)
router = ModelRouter()
print("=" * 50)
print("✅ Model Router initialized!")

## ಸೇವೆ ಆರೋಗ್ಯ ಪರಿಶೀಲನೆ

ನಮ್ಮ ಫೌಂಡ್ರಿ ಲೋಕಲ್ ಸೇವೆಯ ಆರೋಗ್ಯವನ್ನು ಪರಿಶೀಲಿಸಿ ಯಾವ ಮಾದರಿಗಳು ಲಭ್ಯವಿವೆ ಎಂದು ನೋಡೋಣ:


In [ ]:
# Check service health and available models
health = router.check_service_health()

print("🏥 **Service Health Check**")
print("=" * 50)
print(f"🚦 Status: {health['status']}")
print(f"🔗 Base URL: {health['base_url']}")

if health['status'] == 'healthy':
    print(f"\n📋 **Available Models:**")
    for i, model in enumerate(health['available_models'], 1):
        print(f"   {i}. {model}")
    
    print(f"\n🎯 **Configured Models:**")
    for tool, config in router.tools.items():
        model = config['model']
        status = "✅ Available" if model in health['available_models'] else "❌ Not Available"
        print(f"   {tool.title()}: {model} - {status}")
    
    available_tools = len(health['available_configured'])
    total_tools = len(health['configured_models'])
    print(f"\n📊 **Tools Ready:** {available_tools}/{total_tools}")
    
    if available_tools < total_tools:
        missing_models = set(health['configured_models']) - set(health['available_configured'])
        print(f"⚠️ **Missing Models:** {', '.join(missing_models)}")
        print("💡 To start missing models, run: foundry model run <model-name>")
else:
    print(f"❌ **Error:** {health.get('error', 'Unknown error')}")
    print("\n🔧 **Troubleshooting:**")
    print("1. Ensure Foundry Local is running: foundry service status")
    print("2. Start a model: foundry model run phi-4-mini")
    print("3. Check the endpoint URL is correct")

## ವರ್ಗೀಕರಣ ಪರೀಕ್ಷೆ

ನಾವು ರೌಟರ್‌ನ ವಿವಿಧ ರೀತಿಯ ಪ್ರಶ್ನೆಗಳನ್ನು ವರ್ಗೀಕರಿಸುವ ಸಾಮರ್ಥ್ಯವನ್ನು ಪರೀಕ್ಷಿಸೋಣ:


In [ ]:
def test_classification(test_queries: List[str]):
    """Test the classification system with various queries."""
    print("🔍 **Classification Testing**")
    print("=" * 60)
    
    for i, query in enumerate(test_queries, 1):
        tool = router.select_tool(query)
        tool_config = router.tools[tool]
        print(f"\n{i}. **Query:** {query}")
        print(f"   🎯 **Classified as:** {tool.title()}")
        print(f"   🤖 **Model:** {tool_config['model']}")
        print(f"   🌡️ **Temperature:** {tool_config['temperature']}")
        print(f"   📝 **Purpose:** {tool_config['notes']}")

# Test classification with diverse queries
test_queries = [
    # General queries
    "What is artificial intelligence?",
    "Tell me about Microsoft Foundry Local",
    
    # Code queries
    "Write a Python function to sort a list",
    "Fix this bug in my JavaScript code",
    "How do I implement a binary search algorithm?",
    
    # Reasoning queries
    "Why is edge AI becoming important?",
    "Explain step-by-step how neural networks work",
    "Compare the pros and cons of local vs cloud inference",
    
    # Creative queries
    "Write a short story about robots",
    "Create a poem about technology",
    "Write dialogue for a sci-fi movie"
]

test_classification(test_queries)

## ಲೈವ್ ರೌಟಿಂಗ್ ಉದಾಹರಣೆಗಳು

ಈಗ ನಿಜವಾದ ಪ್ರತಿಕ್ರಿಯೆಗಳೊಂದಿಗೆ ರೌಟರ್ ಅನ್ನು ಕ್ರಿಯಾಶೀಲವಾಗಿ ನೋಡೋಣ:


In [ ]:
def demonstrate_routing(example_queries: List[str]):
    """Demonstrate live routing with actual model responses."""
    print("🚀 **Live Routing Demonstration**")
    print("=" * 70)
    
    for i, query in enumerate(example_queries, 1):
        print(f"\n{'='*70}")
        print(f"Example {i}: {query}")
        print(f"{'='*70}")
        
        # Route and get response
        result = router.route_and_run(query)
        
        # Display routing information
        print(f"🎯 **Tool Selected:** {result['tool'].title()}")
        print(f"🤖 **Model Used:** {result['model']}")
        print(f"🌡️ **Temperature:** {result['temperature']}")
        print(f"⏱️ **Processing Time:** {result['processing_time']:.2f}s")
        print(f"📝 **Tool Purpose:** {result['tool_description']}")
        
        # Display response
        print(f"\n💬 **Response:**")
        print(result['answer'])

# Example queries for live demonstration
demo_queries = [
    "What are the main benefits of running AI models locally?",
    "Write a Python function to calculate fibonacci numbers",
    "Explain step-by-step why local AI inference is faster than cloud",
    "Write a creative story about an AI assistant living on an edge device"
]

demonstrate_routing(demo_queries)

## ಕಾರ್ಯಕ್ಷಮತೆ ಹೋಲಿಕೆ

ನಾವು ಒಂದೇ ಕಾರ್ಯದಲ್ಲಿ ವಿಭಿನ್ನ ಮಾದರಿಗಳು ಹೇಗೆ ಕಾರ್ಯನಿರ್ವಹಿಸುತ್ತವೆ ಎಂದು ಹೋಲಿಸೋಣ:


In [ ]:
def compare_model_performance(test_prompt: str, max_tokens: int = 150):
    """Compare how different models handle the same prompt."""
    print(f"⚖️ **Model Performance Comparison**")
    print(f"📝 **Test Prompt:** {test_prompt}")
    print("=" * 70)
    
    # Test with each configured model
    for tool_name, tool_config in router.tools.items():
        model = tool_config['model']
        temperature = tool_config['temperature']
        
        print(f"\n🔧 **{tool_name.title()} Tool ({model})**")
        print(f"   🌡️ Temperature: {temperature}")
        print("-" * 50)
        
        start_time = time.time()
        response = router.chat(
            model=model, 
            content=test_prompt, 
            max_tokens=max_tokens, 
            temperature=temperature
        )
        end_time = time.time()
        
        processing_time = end_time - start_time
        
        print(f"⏱️ **Time:** {processing_time:.2f}s")
        print(f"💬 **Response:** {response}")
        
        # Calculate response length
        response_length = len(response.split())
        print(f"📏 **Length:** {response_length} words")
        
        if "Error" in response:
            print("❌ **Status:** Error")
        else:
            print("✅ **Status:** Success")

# Test with a neutral prompt that could work for any model
comparison_prompt = "Explain the concept of edge computing in simple terms."
compare_model_performance(comparison_prompt)

## ಇಂಟರಾಕ್ಟಿವ್ ರೌಟರ್ ಪರೀಕ್ಷೆ

ನಿಮ್ಮ ಸ್ವಂತ ಕಸ್ಟಮ್ ಪ್ರಶ್ನೆಗಳೊಂದಿಗೆ ರೌಟರ್ ಅನ್ನು ಪರೀಕ್ಷಿಸಿ:


In [ ]:
def interactive_test(custom_query: str, force_tool: str = None):
    """Test the router with a custom query, optionally forcing a specific tool."""
    print(f"🎪 **Interactive Router Test**")
    print(f"💭 **Your Query:** {custom_query}")
    print("=" * 60)
    
    if force_tool:
        if force_tool not in router.tools:
            print(f"❌ Invalid tool: {force_tool}. Available: {list(router.tools.keys())}")
            return
        selected_tool = force_tool
        print(f"🔧 **Forced Tool:** {selected_tool.title()}")
    else:
        selected_tool = router.select_tool(custom_query)
        print(f"🎯 **Auto-Selected Tool:** {selected_tool.title()}")
    
    # Get full result
    if force_tool:
        # Manual routing
        tool_config = router.tools[force_tool]
        model = tool_config['model']
        temperature = tool_config['temperature']
        
        start_time = time.time()
        answer = router.chat(model, custom_query, temperature=temperature)
        end_time = time.time()
        
        result = {
            'tool': force_tool,
            'model': model,
            'temperature': temperature,
            'processing_time': end_time - start_time,
            'answer': answer
        }
    else:
        # Automatic routing
        result = router.route_and_run(custom_query)
    
    # Display results
    print(f"\n📊 **Routing Details:**")
    print(f"   🔧 Tool: {result['tool'].title()}")
    print(f"   🤖 Model: {result['model']}")
    print(f"   🌡️ Temperature: {result['temperature']}")
    print(f"   ⏱️ Time: {result['processing_time']:.2f}s")
    
    print(f"\n💬 **Response:**")
    print(result['answer'])
    
    return result

# Test with your custom query
custom_query = "How can I optimize my Python code for better performance?"
# Uncomment the next line to force a specific tool
# force_tool = "code"  # Options: "general", "reasoning", "code", "creative"
force_tool = None

result = interactive_test(custom_query, force_tool=force_tool)

## ಉನ್ನತ ಸಂರಚನೆ

ರೌಟರ್ ಸಂರಚನೆಯನ್ನು ಹೇಗೆ ಕಸ್ಟಮೈಸ್ ಮಾಡುವುದು ಎಂಬುದನ್ನು ಪ್ರದರ್ಶಿಸಿ:


In [ ]:
class CustomModelRouter(ModelRouter):
    """Extended router with custom configuration and additional features."""
    
    def __init__(self, custom_tools: Dict[str, Dict[str, Any]] = None):
        self.custom_tools = custom_tools
        super().__init__()
    
    def _load_tool_registry(self) -> Dict[str, Dict[str, Any]]:
        """Load custom tool registry if provided."""
        if self.custom_tools:
            return self.custom_tools
        return super()._load_tool_registry()
    
    def add_custom_classifier(self, tool_name: str, keywords: List[str]):
        """Add a custom classification rule."""
        self.custom_keywords = getattr(self, 'custom_keywords', {})
        self.custom_keywords[tool_name] = keywords
    
    def select_tool(self, user_query: str) -> str:
        """Enhanced tool selection with custom rules."""
        # Check custom keywords first
        if hasattr(self, 'custom_keywords'):
            query_lower = user_query.lower()
            for tool_name, keywords in self.custom_keywords.items():
                if any(keyword in query_lower for keyword in keywords):
                    return tool_name
        
        # Fall back to default classification
        return super().select_tool(user_query)
    
    def get_tool_statistics(self) -> Dict[str, Any]:
        """Get statistics about tool usage."""
        if not hasattr(self, 'usage_stats'):
            self.usage_stats = {tool: 0 for tool in self.tools.keys()}
            self.total_requests = 0
        
        return {
            'usage_by_tool': self.usage_stats.copy(),
            'total_requests': self.total_requests,
            'most_used_tool': max(self.usage_stats, key=self.usage_stats.get) if self.usage_stats else None
        }
    
    def route_and_run(self, prompt: str) -> Dict[str, Any]:
        """Enhanced routing with usage tracking."""
        result = super().route_and_run(prompt)
        
        # Track usage
        if not hasattr(self, 'usage_stats'):
            self.usage_stats = {tool: 0 for tool in self.tools.keys()}
            self.total_requests = 0
        
        self.usage_stats[result['tool']] += 1
        self.total_requests += 1
        
        return result

# Example: Create a router with custom configuration
custom_config = {
    "general": {
        "model": "phi-4-mini",
        "notes": "General purpose model",
        "temperature": 0.7
    },
    "technical": {
        "model": "qwen2.5-7b",
        "notes": "Technical documentation and analysis",
        "temperature": 0.3
    },
    "creative": {
        "model": "phi-4-mini",
        "notes": "Creative writing and storytelling",
        "temperature": 0.9
    }
}

print("🔧 **Creating Custom Router Configuration**")
custom_router = CustomModelRouter(custom_tools=custom_config)

# Add custom classification rules
custom_router.add_custom_classifier("technical", ["documentation", "specification", "architecture", "design"])

print("✅ Custom router created with enhanced features")
print(f"🎯 Available tools: {list(custom_router.tools.keys())}")

## ಬ್ಯಾಚ್ ಪ್ರೊಸೆಸಿಂಗ್ ಉದಾಹರಣೆ

ಬಹು ಪ್ರಶ್ನೆಗಳನ್ನು ಪರಿಣಾಮಕಾರಿಯಾಗಿ ಪ್ರಕ್ರಿಯೆಗೊಳಿಸಿ:


In [ ]:
def batch_process_queries(queries: List[str], router_instance: ModelRouter = None) -> List[Dict[str, Any]]:
    """Process multiple queries in batch and analyze results."""
    if router_instance is None:
        router_instance = router
    
    print(f"📦 **Batch Processing {len(queries)} Queries**")
    print("=" * 60)
    
    results = []
    total_time = 0
    tool_usage = {}
    
    for i, query in enumerate(queries, 1):
        print(f"\n{i}. Processing: {query[:50]}{'...' if len(query) > 50 else ''}")
        
        result = router_instance.route_and_run(query)
        results.append(result)
        
        # Track statistics
        total_time += result['processing_time']
        tool = result['tool']
        tool_usage[tool] = tool_usage.get(tool, 0) + 1
        
        print(f"   🎯 Tool: {tool.title()} | ⏱️ {result['processing_time']:.2f}s")
    
    # Display summary
    print(f"\n📊 **Batch Processing Summary**")
    print("=" * 40)
    print(f"📝 Total Queries: {len(queries)}")
    print(f"⏱️ Total Time: {total_time:.2f}s")
    print(f"🚀 Average Time: {total_time/len(queries):.2f}s per query")
    
    print(f"\n🎯 **Tool Usage:**")
    for tool, count in sorted(tool_usage.items()):
        percentage = (count / len(queries)) * 100
        print(f"   {tool.title()}: {count} queries ({percentage:.1f}%)")
    
    return results

# Batch test queries
batch_queries = [
    "What is machine learning?",
    "Write a Python function for quicksort",
    "Why is data privacy important in AI?",
    "Create a haiku about artificial intelligence",
    "How do I debug a memory leak in my application?",
    "Explain the difference between supervised and unsupervised learning",
    "Write a short story about a robot chef",
    "What are the advantages of edge computing?"
]

batch_results = batch_process_queries(batch_queries)

# Show detailed results for first few queries
print(f"\n📋 **Sample Detailed Results:**")
for i, result in enumerate(batch_results[:3], 1):
    print(f"\n{i}. **Tool:** {result['tool'].title()} | **Model:** {result['model']}")
    print(f"   **Response:** {result['answer'][:100]}{'...' if len(result['answer']) > 100 else ''}")

## ಉತ್ಪಾದನೆ ಮೇಲ್ವಿಚಾರಣೆ

ಉತ್ಪಾದನೆ-ಸಿದ್ಧ ಮೇಲ್ವಿಚಾರಣೆ ಮತ್ತು ಲಾಗಿಂಗ್ ಉದಾಹರಣೆ:


In [ ]:
class ProductionModelRouter(ModelRouter):
    """Production-ready router with comprehensive monitoring."""
    
    def __init__(self):
        super().__init__()
        self.metrics = {
            'total_requests': 0,
            'successful_requests': 0,
            'failed_requests': 0,
            'total_processing_time': 0,
            'tool_usage': {},
            'model_performance': {},
            'error_log': []
        }
    
    def route_and_run(self, prompt: str) -> Dict[str, Any]:
        """Enhanced routing with comprehensive monitoring."""
        self.metrics['total_requests'] += 1
        request_id = self.metrics['total_requests']
        
        try:
            result = super().route_and_run(prompt)
            
            # Track success metrics
            self.metrics['successful_requests'] += 1
            self.metrics['total_processing_time'] += result['processing_time']
            
            # Track tool usage
            tool = result['tool']
            if tool not in self.metrics['tool_usage']:
                self.metrics['tool_usage'][tool] = {'count': 0, 'total_time': 0}
            self.metrics['tool_usage'][tool]['count'] += 1
            self.metrics['tool_usage'][tool]['total_time'] += result['processing_time']
            
            # Track model performance
            model = result['model']
            if model not in self.metrics['model_performance']:
                self.metrics['model_performance'][model] = {'requests': 0, 'total_time': 0, 'avg_time': 0}
            self.metrics['model_performance'][model]['requests'] += 1
            self.metrics['model_performance'][model]['total_time'] += result['processing_time']
            self.metrics['model_performance'][model]['avg_time'] = (
                self.metrics['model_performance'][model]['total_time'] / 
                self.metrics['model_performance'][model]['requests']
            )
            
            # Add monitoring metadata
            result['request_id'] = request_id
            result['status'] = 'success'
            
            return result
            
        except Exception as e:
            # Track failure metrics
            self.metrics['failed_requests'] += 1
            error_entry = {
                'request_id': request_id,
                'timestamp': time.strftime("%Y-%m-%d %H:%M:%S"),
                'prompt': prompt[:100],  # Truncate for privacy
                'error': str(e)
            }
            self.metrics['error_log'].append(error_entry)
            
            return {
                'request_id': request_id,
                'status': 'error',
                'error': str(e),
                'timestamp': time.strftime("%Y-%m-%d %H:%M:%S")
            }
    
    def get_performance_report(self) -> Dict[str, Any]:
        """Generate comprehensive performance report."""
        total_requests = self.metrics['total_requests']
        if total_requests == 0:
            return {'message': 'No requests processed yet'}
        
        success_rate = (self.metrics['successful_requests'] / total_requests) * 100
        avg_processing_time = (
            self.metrics['total_processing_time'] / max(1, self.metrics['successful_requests'])
        )
        
        return {
            'overview': {
                'total_requests': total_requests,
                'successful_requests': self.metrics['successful_requests'],
                'failed_requests': self.metrics['failed_requests'],
                'success_rate': f"{success_rate:.1f}%",
                'average_processing_time': f"{avg_processing_time:.2f}s"
            },
            'tool_usage': self.metrics['tool_usage'],
            'model_performance': self.metrics['model_performance'],
            'recent_errors': self.metrics['error_log'][-5:] if self.metrics['error_log'] else []
        }

# Create production router and test
print("🏭 **Production Router Testing**")
prod_router = ProductionModelRouter()

# Process several requests
test_requests = [
    "Explain quantum computing",
    "Write a function to reverse a string",
    "Why is cybersecurity important?",
    "Create a poem about the future"
]

print(f"\nProcessing {len(test_requests)} test requests...")
for i, request in enumerate(test_requests, 1):
    result = prod_router.route_and_run(request)
    status = "✅" if result['status'] == 'success' else "❌"
    print(f"{i}. {status} Request {result['request_id']}: {request[:30]}...")

# Generate performance report
report = prod_router.get_performance_report()

print(f"\n📊 **Performance Report**")
print("=" * 50)
print(f"📈 **Overview:**")
for key, value in report['overview'].items():
    print(f"   {key.replace('_', ' ').title()}: {value}")

print(f"\n🎯 **Tool Usage:**")
for tool, stats in report['tool_usage'].items():
    avg_time = stats['total_time'] / stats['count']
    print(f"   {tool.title()}: {stats['count']} requests (avg: {avg_time:.2f}s)")

print(f"\n🤖 **Model Performance:**")
for model, stats in report['model_performance'].items():
    print(f"   {model}: {stats['requests']} requests (avg: {stats['avg_time']:.2f}s)")

if report['recent_errors']:
    print(f"\n❌ **Recent Errors:**")
    for error in report['recent_errors']:
        print(f"   Request {error['request_id']}: {error['error']}")
else:
    print(f"\n✅ **No Recent Errors**")

## ಸಾರಾಂಶ ಮತ್ತು ಉತ್ತಮ ಅಭ್ಯಾಸಗಳು

ಈ ನೋಟ್ಬುಕ್ ಒಂದು ಸುಧಾರಿತ ಬುದ್ಧಿವಂತಿಕೆಯ ಮಾದರಿ ಮಾರ್ಗದರ್ಶನ ವ್ಯವಸ್ಥೆಯನ್ನು ಪ್ರದರ್ಶಿಸಿದೆ:

### ✅ ಪ್ರಮುಖ ವೈಶಿಷ್ಟ್ಯಗಳು ಪ್ರದರ್ಶಿಸಲಾಗಿದೆ

1. **🎯 ಬುದ್ಧಿವಂತಿಕೆಯ ವರ್ಗೀಕರಣ**: ಕೀವರ್ಡ್ ವಿಶ್ಲೇಷಣೆಯ ಮೂಲಕ ಸ್ವಯಂಚಾಲಿತ ಕಾರ್ಯ ವರ್ಗೀಕರಣ
2. **🔧 ಡೈನಾಮಿಕ್ ಮಾದರಿ ಆಯ್ಕೆ**: ವಿಶೇಷ ಮಾದರಿಗಳಿಗೆ ಪ್ರಶ್ನೆಗಳನ್ನು ಮಾರ್ಗದರ್ಶನ ಮಾಡುವುದು
3. **⚙️ ಲವಚಿಕ ಸಂರಚನೆ**: ಪರಿಸರ ಆಧಾರಿತ ಮತ್ತು ಕಸ್ಟಮ್ ಟೂಲ್ ರಿಜಿಸ್ಟ್ರಿಗಳು
4. **📊 ಆರೋಗ್ಯ ಮಾನಿಟರಿಂಗ್**: ಸೇವೆ ಮತ್ತು ಮಾದರಿ ಲಭ್ಯತೆ ಪರಿಶೀಲನೆಗಳು
5. **⚡ ಕಾರ್ಯಕ್ಷಮತೆ ಟ್ರ್ಯಾಕಿಂಗ್**: ಪ್ರತಿಕ್ರಿಯೆ ಸಮಯ ಮತ್ತು ಬಳಕೆ ವಿಶ್ಲೇಷಣೆಗಳು
6. **🏭 ಉತ್ಪಾದನೆಗೆ ಸಿದ್ಧ**: ಸಮಗ್ರ ಮಾನಿಟರಿಂಗ್ ಮತ್ತು ದೋಷ ನಿರ್ವಹಣೆ

### 🎨 ಮಾದರಿ ವರ್ಗಗಳ ಸಾರಾಂಶ

| ವರ್ಗ | ಡೀಫಾಲ್ಟ್ ಮಾದರಿ | ತಾಪಮಾನ | ಉತ್ತಮಕ್ಕಾಗಿ |
|----------|---------------|-------------|----------|
| **🌐 ಸಾಮಾನ್ಯ** | phi-4-mini | 0.7 | ಪ್ರಶ್ನೋತ್ತರ, ಚಾಟ್, ಸಾಮಾನ್ಯ ಕಾರ್ಯಗಳು |
| **🧠 ತರ್ಕಶಕ್ತಿ** | deepseek-r1-7b | 0.3 | ವಿಶ್ಲೇಷಣೆ, ವಿವರಣೆಗಳು |
| **💻 ಕೋಡ್** | qwen2.5-7b | 0.2 | ಪ್ರೋಗ್ರಾಮಿಂಗ್, ಡಿಬಗಿಂಗ್ |
| **🎨 ಸೃಜನಶೀಲ** | phi-4-mini | 0.9 | ಕಥೆಗಳು, ಕವಿತೆಗಳು, ಸೃಜನಶೀಲ ಬರವಣಿಗೆ |

### 🔍 ವರ್ಗೀಕರಣ ನಿಯಮಗಳು

- **ಕೋಡ್ ಪತ್ತೆ**: `code`, `python`, `function`, `class`, `bug`, `debug`, `programming`
- **ತರ್ಕಶಕ್ತಿ ಪತ್ತೆ**: `why`, `how`, `explain`, `step-by-step`, `reason`, `analyze`
- **ಸೃಜನಶೀಲ ಪತ್ತೆ**: `story`, `poem`, `creative`, `imagine`, `write`, `tale`
- **ಡೀಫಾಲ್ಟ್**: ಇತರೆ ಎಲ್ಲಾ ಪ್ರಶ್ನೆಗಳು ಸಾಮಾನ್ಯ ಮಾದರಿಯನ್ನು ಬಳಸುತ್ತವೆ

### 💡 ಉತ್ತಮ ಅಭ್ಯಾಸಗಳು

1. **🎯 ಮಾದರಿ ವಿಶೇಷೀಕರಣ**: ಪ್ರತಿ ಮಾದರಿಯ ಶಕ್ತಿಗಳಿಗೆ ಅನುಗುಣವಾಗಿ ಬಳಸುವುದು
2. **🌡️ ತಾಪಮಾನ ಹೊಂದಿಕೆ**: ನಿಖರತೆಗಾಗಿ ಕಡಿಮೆ, ಸೃಜನಶೀಲತೆಗೆ ಹೆಚ್ಚು
3. **📊 ನಿರಂತರ ಮಾನಿಟರಿಂಗ್**: ಕಾರ್ಯಕ್ಷಮತೆ ಮತ್ತು ಬಳಕೆ ಮಾದರಿಗಳನ್ನು ಟ್ರ್ಯಾಕ್ ಮಾಡುವುದು
4. **🔄 ಬ್ಯಾಕಪ್ ತಂತ್ರಗಳು**: ಮಾದರಿಗಳು ಲಭ್ಯವಿಲ್ಲದಾಗ ಸೌಮ್ಯ ಕುಸಿತ
5. **⚖️ ಲೋಡ್ ಸಮತೋಲನ**: ಹಲವಾರು ಉದಾಹರಣೆಗಳ ನಡುವೆ ಲೋಡ್ ಹಂಚಿಕೆ
6. **🔧 ಕಸ್ಟಮ್ ವರ್ಗೀಕರಣ**: ಕ್ಷೇತ್ರ-ನಿರ್ದಿಷ್ಟ ಮಾರ್ಗದರ್ಶನ ನಿಯಮಗಳನ್ನು ಸೇರಿಸುವುದು

### 🚀 ಬಳಕೆ ಪ್ರಕರಣಗಳು

- **👨‍💻 ಅಭಿವೃದ್ಧಿ ಉಪಕರಣಗಳು**: ಬುದ್ಧಿವಂತಿಕೆಯ ಮಾದರಿ ಮಾರ್ಗದರ್ಶನದೊಂದಿಗೆ IDE ಪ್ಲಗಿನ್‌ಗಳು
- **🎧 ಗ್ರಾಹಕ ಬೆಂಬಲ**: ವಿಶೇಷ ಬೆಂಬಲ ಮಾದರಿಗಳಿಗೆ ಪ್ರಶ್ನೆಗಳನ್ನು ಮಾರ್ಗದರ್ಶನ ಮಾಡುವುದು
- **📝 ವಿಷಯ ಸೃಷ್ಟಿ**: ಕಾರ್ಯಗಳನ್ನು ಸೂಕ್ತ ಸೃಜನಶೀಲ ಮಾದರಿಗಳಿಗೆ ಹೊಂದಿಸುವುದು
- **📚 ಡಾಕ್ಯುಮೆಂಟೇಶನ್**: ತಾಂತ್ರಿಕ ಬರವಣಿಗೆ ಮಾದರಿಗಳಿಗೆ ಮಾರ್ಗದರ್ಶನ
- **🎓 ಶಿಕ್ಷಣ ವೇದಿಕೆಗಳು**: ವಿಷಯ-ನಿರ್ದಿಷ್ಟ ಮಾದರಿ ಮಾರ್ಗದರ್ಶನ

### 🔮 ಮುಂದಿನ ಹಂತಗಳು

- **🤖 ಯಂತ್ರ ಅಧ್ಯಯನ**: ಉತ್ತಮ ವರ್ಗೀಕರಣಕ್ಕಾಗಿ ML ಬಳಕೆ
- **🔧 ಫಂಕ್ಷನ್ ಕರೆ**: ನಿರ್ದಿಷ್ಟ ಟೂಲ್ ಸಾಮರ್ಥ್ಯಗಳೊಂದಿಗೆ ಮಾದರಿಗಳಿಗೆ ಮಾರ್ಗದರ್ಶನ
- **📸 ಬಹು-ಮಾದರಿ**: ಇನ್‌ಪುಟ್ ಪ್ರಕಾರ (ಪಠ್ಯ, ಚಿತ್ರ, ಧ್ವನಿ) ಆಧಾರಿತ ಮಾರ್ಗದರ್ಶನ
- **🌐 ಫೆಡರೇಟೆಡ್ ಮಾರ್ಗದರ್ಶನ**: ಹಲವಾರು Foundry Local ಉದಾಹರಣೆಗಳ ನಡುವೆ ಮಾರ್ಗದರ್ಶನ
- **💰 ವೆಚ್ಚ ಆಪ್ಟಿಮೈಜೆಷನ್**: ವೆಚ್ಚ ಮತ್ತು ಕಾರ್ಯಕ್ಷಮತೆ ಮೌಲ್ಯಮಾಪನ ಆಧಾರಿತ ಮಾರ್ಗದರ್ಶನ

ಈ ಬುದ್ಧಿವಂತಿಕೆಯ ಮಾರ್ಗದರ್ಶನ ವ್ಯವಸ್ಥೆ ಹಲವಾರು ವಿಶೇಷ ಮಾದರಿಗಳ ಮೌಲ್ಯವನ್ನು ಗರಿಷ್ಠಗೊಳಿಸುವುದನ್ನು ತೋರಿಸುತ್ತದೆ, Microsoft Foundry Local ನೊಂದಿಗೆ ಸ್ಥಳೀಯ ನಿರ್ಣಯದ ಗೌಪ್ಯತೆ ಮತ್ತು ಕಾರ್ಯಕ್ಷಮತೆ ಲಾಭಗಳನ್ನು ಕಾಯ್ದುಕೊಳ್ಳುತ್ತಾ. "ಮಾದರಿಗಳನ್ನು ಉಪಕರಣಗಳಾಗಿ" ಪರಿಕಲ್ಪನೆ ಪ್ರತಿ ನಿರ್ದಿಷ್ಟ ಕಾರ್ಯಕ್ಕೆ ಅತ್ಯುತ್ತಮ ಮಾದರಿಯನ್ನು ಸ್ವಯಂಚಾಲಿತವಾಗಿ ಆಯ್ಕೆಮಾಡುವ ಸುಧಾರಿತ AI ಅಪ್ಲಿಕೇಶನ್‌ಗಳನ್ನು ನಿರ್ಮಿಸಲು ಸಾಧ್ಯವಾಗಿಸುತ್ತದೆ.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ಅಸ್ವೀಕಾರ**:  
ಈ ದಸ್ತಾವೇಜು AI ಅನುವಾದ ಸೇವೆ [Co-op Translator](https://github.com/Azure/co-op-translator) ಬಳಸಿ ಅನುವಾದಿಸಲಾಗಿದೆ. ನಾವು ನಿಖರತೆಯಿಗಾಗಿ ಪ್ರಯತ್ನಿಸುತ್ತಿದ್ದರೂ, ಸ್ವಯಂಚಾಲಿತ ಅನುವಾದಗಳಲ್ಲಿ ತಪ್ಪುಗಳು ಅಥವಾ ಅಸತ್ಯತೆಗಳು ಇರಬಹುದು ಎಂದು ದಯವಿಟ್ಟು ಗಮನಿಸಿ. ಮೂಲ ಭಾಷೆಯಲ್ಲಿರುವ ಮೂಲ ದಸ್ತಾವೇಜನ್ನು ಅಧಿಕೃತ ಮೂಲವೆಂದು ಪರಿಗಣಿಸಬೇಕು. ಮಹತ್ವದ ಮಾಹಿತಿಗಾಗಿ, ವೃತ್ತಿಪರ ಮಾನವ ಅನುವಾದವನ್ನು ಶಿಫಾರಸು ಮಾಡಲಾಗುತ್ತದೆ. ಈ ಅನುವಾದ ಬಳಕೆಯಿಂದ ಉಂಟಾಗುವ ಯಾವುದೇ ತಪ್ಪು ಅರ್ಥಮಾಡಿಕೊಳ್ಳುವಿಕೆ ಅಥವಾ ತಪ್ಪು ವಿವರಣೆಗಳಿಗೆ ನಾವು ಹೊಣೆಗಾರರಾಗುವುದಿಲ್ಲ.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
